# Reflection


In the context of LLM agent building, reflection refers to the process of prompting an LLM to observe its past steps (along with potential observations from tools/the environment) to assess the quality of the chosen actions.
This is then used downstream for things like re-planning, search, or evaluation.

![Reflection](./img/reflection.png)

This notebook demonstrates a very simple form of reflection in LangGraph.

#### Prerequisites

We will be using a basic agent with a search tool here.

In [1]:
%pip install -U --quiet  langgraph langchain-fireworks
%pip install -U --quiet tavily-python

In [2]:
import getpass
import os


def _set_if_undefined(var: str) -> None:
    if os.environ.get(var):
        return
    os.environ[var] = getpass.getpass(var)


# Optional: Configure tracing to visualize and debug the agent
_set_if_undefined("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Reflection"

_set_if_undefined("FIREWORKS_API_KEY")

## Generate

For our example, we will create a "5 paragraph essay" generator. First, create the generator:


In [3]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_fireworks import ChatFireworks

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an essay assistant tasked with writing excellent 5-paragraph essays."
            " Generate the best essay possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
llm = ChatFireworks(
    model="accounts/fireworks/models/mixtral-8x7b-instruct",
    model_kwargs={"max_tokens": 32768},
)
generate = prompt | llm

In [4]:
essay = ""
request = HumanMessage(
    content="Write an essay on why the little prince is relevant in modern childhood"
)
for chunk in generate.stream({"messages": [request]}):
    print(chunk.content, end="")
    essay += chunk.content

Title: The Relevance of The Little Prince in Modern Childhood

The Little Prince, a novella by Antoine de Saint-Exupéry, has been a childhood favorite for generations. Despite being published over seven decades ago, its timeless themes continue to resonate with modern children, making it highly relevant in contemporary childhood.

Firstly, the story explores the complex nature of human relationships, which is particularly relevant for modern children growing up in an increasingly connected yet impersonal world. Through the little prince's encounters with various grown-ups on different planets, the book highlights the importance of genuine connections and understanding. In an age where digital communication often replaces face-to-face interaction, this message is more pertinent than ever. The Little Prince encourages children to look beyond superficial relationships and seek deeper connections, fostering empathy and emotional intelligence.

Secondly, the book deals with the concept of r

### Reflect

In [5]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations, including requests for length, depth, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm

In [6]:
reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=essay)]}):
    print(chunk.content, end="")
    reflection += chunk.content

Essay Grade: B+

The essay you submitted provides a clear and well-structured argument about the relevance of The Little Prince in modern childhood. You have demonstrated a strong understanding of the text and its themes, and have effectively applied them to the context of contemporary childhood. However, there are some areas where improvement could be made to enhance the depth, style, and overall flow of your essay.

1. Length: While your essay is well-written and informative, it is relatively brief. Expanding on each point with more detailed analysis and examples would strengthen your argument and demonstrate a more comprehensive understanding of the text. Aim for a minimum of 500 words to allow for a more in-depth exploration of your ideas.

2. Depth: Although you have touched upon the relevance of the novel's themes, further analysis is needed to truly establish its significance in modern childhood. For example, when discussing the complex nature of human relationships, delve into 

### Repeat

And... that's all there is too it! You can repeat in a loop for a fixed number of steps, or use an LLM (or other check) to decide when the finished product is good enough.

In [7]:
for chunk in generate.stream(
    {"messages": [request, AIMessage(content=essay), HumanMessage(content=reflection)]}
):
    print(chunk.content, end="")

Title: The Relevance of The Little Prince in Modern Childhood: A Contemporary Analysis

In the digital age, where human connections are often overshadowed by virtual communication, Antoine de Saint-Exupéry's The Little Prince remains a timeless classic that offers invaluable insights for modern children. This essay aims to delve deeper into the relevance of this novella in contemporary childhood, focusing on the complex nature of human relationships, self-discovery, and the inevitability of loss.

Firstly, The Little Prince offers a powerful critique of the superficiality that permeates the digital world. Through the little prince's encounters with various grown-ups, the book emphasizes the importance of genuine connections and understanding. Despite being published in 1943, Saint-Exupéry's work uncannily predicts the isolating effects of technology on human interaction. It encourages children to seek deeper connections, fostering empathy and emotional intelligence. For instance, the l

## Define graph

Now that we've shown each step in isolation, we can wire it up in a graph.

In [8]:
from typing import List, Sequence

from langgraph.graph import END, MessageGraph


async def generation_node(state: Sequence[BaseMessage]):
    return await generate.ainvoke({"messages": state})


async def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [messages[0]] + [
        cls_map[msg.type](content=msg.content) for msg in messages[1:]
    ]
    res = await reflect.ainvoke({"messages": translated})
    # We treat the output of this as human feedback for the generator
    return HumanMessage(content=res.content)


builder = MessageGraph()
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_edge("__start__", "generate")


def should_continue(state: List[BaseMessage]):
    if len(state) > 6:
        # End after 3 iterations
        return END
    return "reflect"


builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
graph = builder.compile()

In [9]:
async for event in graph.astream(
    [
        HumanMessage(
            content="Generate an essay on the topicality of The Little Prince and its message in modern life"
        )
    ],
):
    print(event)
    print("---")

{'generate': AIMessage(content="Title: The Enduring Relevance of The Little Prince: A Timeless Message for Modern Life\n\nIntroduction:\nAntoine de Saint-Exupéry's The Little Prince is a canonical work of literature, beloved by generations since its publication in 1943. The novella has been translated into more than 250 languages and sold over 140 million copies, making it one of the best-selling books of all time. Its enchanting story transcends cultural boundaries and continues to captivate audiences of all ages. The Little Prince's timeless message remains relevant in modern life, offering insightful commentary on themes such as love, loneliness, responsibility, and the superficiality of the adult world. In this essay, we will discuss the topicality of The Little Prince and its enduring message in today's fast-paced, digitally-connected society.\n\nBody Paragraph 1 - Love and Loneliness:\nOne of the most enduring aspects of The Little Prince is its exploration of love and relationsh

In [10]:
ChatPromptTemplate.from_messages(event[END]).pretty_print()

================================ Human Message =================================

Generate an essay on the topicality of The Little Prince and its message in modern life

================================== Ai Message ==================================

Title: The Enduring Relevance of The Little Prince: A Timeless Message for Modern Life

Introduction:
Antoine de Saint-Exupéry's The Little Prince is a canonical work of literature, beloved by generations since its publication in 1943. The novella has been translated into more than 250 languages and sold over 140 million copies, making it one of the best-selling books of all time. Its enchanting story transcends cultural boundaries and continues to captivate audiences of all ages. The Little Prince's timeless message remains relevant in modern life, offering insightful commentary on themes such as love, loneliness, responsibility, and the superficiality of the adult world. In this essay, we will discuss the topicality of The Little Princ

## Conclusion

Now that you've applied reflection to an LLM agent, I'll note one thing: self-reflection is inherently cyclic: it is much more effective if the reflection step has additional context or feedback (from tool observations, checks, etc.). If, like in the scenario above, the reflection step simply prompts the LLM to reflect on its output, it can still benefit the output quality (since the LLM then has multiple "shots" at getting a good output), but it's less guaranteed.
